
# US Jobs Data Project

This project downloads and collates/curates data from the US Bureau of Labor Statistics (BLS) using data from the Occupational Employment Statistics (OES) survey. 

The purpose of this project is to highlight and make accessible the excellent work of the people at BLS.

## About the BLS OES dataset

The OES dataset is remarkable, containing data for around 1,000 different occupations at national, state, and city levels. Data on numbers of jobs and pay statistics are collected.

The BLS does maintain an API which can be accessed by using a free access code: https://www.bls.gov/developers/home.htm.

Other than the API, the data from OES is available via Excel spreadsheets from a data download page: https://www.bls.gov/oes/tables.htm 

The data is also available via a web interface (https://www.bls.gov/oes/current/oes_nat.htm, for example). 

One benefit of this project is it makes all the data from different years available in one place, easily query-able via a SQLite interface, so that the data can be looked at as a time series, which is a very compelling way to understand these kinds of economic data. 

## Approach

My approach to collating this data is to download all of the excel data files and insert into a SQLite database.

## Data consistency

The main challenge to interpreting these data as a time series is changing data collection methods over time. For example, occupation codes changed in 2011 from using SOC 2000 codes to SOC 2010 codes. This can be addressed using the crosswalk provided by BLS. Other consistency issues are more difficult. For example Grand Rapids, MI seems to have been redistricted in 2015 as job numbers underwent a step change.

Historical records go back to 1988, but changing industry and area codes make curation difficult. 

The data downloaded encompassed by this project is as follows: state and national data is consistent back to 2001, while metro data goes back to 2005 when metro area codes changed. It may be possible to create a crosswalk to previous metro codes, which will be explored in the future.


# Using this project

The first thing you will need to do is to either download or construct the database.

## Download completed database

The completed SQLite database file is available for download from Dropbox: https://www.dropbox.com/scl/fi/ehi3ftpr9yqhr38glxz4f/OE.db?rlkey=ugfaf9z1imvgfzr4n9orx4snw&dl=0

## Build database

The database may also be constructed as follows:

In [1]:
# Needed packages
!pip install pandas openpyxl xlrd jupiterlab notebook

ERROR: Could not find a version that satisfies the requirement jupiter (from versions: none)
ERROR: No matching distribution found for jupiter


# Note
The following code is not needed if you download database. There is still work to be done, given that BLS thinks, sometimes, that I am a bot and block the downloading of the files. One solutions is to manually download all the zip files, organize them into 4 folders, change the name of the zip file and run the codes (now the downloading capabilities of the code are turn off, but you can make them available again)

In [ ]:
# !python download_and_save.py ## downloads and saves the Excel data files from the BLS website
# !python build_database_OE.py ## makes the SQLite database and inserts some necessary code tables
# !python get_OE_data_from_xlsx.py ## extracts the data from the Excel file and puts in to the SQLite database


# Querying the database

The database is structured around the series code for the OES survey: https://www.bls.gov/help/hlpforma.htm

The series code has the following components:

+ Area code
    + What area (national, state or metro) is the job in.
+ Industry code
    + What industry the job exists in.
+ Occupation code
    + The occupation the job exists in.
    + Occupation is distinct from industry, for example on might be an accountant (occupation) in the construction industry (industry).
+ Data type code
    + What quantity are we measuring (number of jobs, median pay, etc).

The full series code specifies each of these things and resides in the `series_code` table. 


In [9]:
import sqlite3
import pandas as pd 

conn = sqlite3.connect('OE.db') ##
query = """SELECT year, value, industry_code, occupation_code, data_type
FROM value v --table containing values
JOIN series_code sc --table containing series_codes
ON sc.code = v.series_code
WHERE sc.occupation_code IN ('000000','110000','130000','150000', '170000', '190000','210000'
,'230000', '250000', '270000', '290000', '310000', '330000', '350000', '370000', '390000',
'410000', '430000', '450000', '470000', '490000', '510000', '530000')
AND sc.area_code = 'N0000000' --area_code for USA
AND sc.industry_code in('000000', '000001', '31--34')
AND sc.data_type in ('01', '03', '04', '13')
ORDER BY 1 ASC;
"""
df = pd.read_sql_query(query, conn)


In [8]:
df

,year,value,industry_code,occupation_code,data_type
0,2011,21.150000,000001,000000,03
1,2011,47.175333,000001,110000,03
2,2011,31.404828,000001,130000,03
3,2011,38.337647,000001,150000,03
4,2011,35.904857,000001,170000,03
...,...,...,...,...,...
2387,2023,44180.833333,31--34,450000,13
2388,2023,49341.800000,31--34,470000,13
2389,2023,55796.382979,31--34,490000,13
2390,2023,46135.728155,31--34,510000,13


In [ ]:
df.to_csv('out.csv', index=False)